# Meridional Overturning Circulation Notebook (z-space)

This notebook plots the global MOC in depth space.

Authors: Jan-Erik Tesdal and John Krasting

In [ ]:
# Development mode: constantly refreshes module code
%load_ext autoreload
%autoreload 2

## Framework Code and Diagnostic Setup

In [ ]:
import esnb
from esnb import NotebookDiagnostic, RequestedVariable, CaseGroup2, nbtools
from esnb.sites.gfdl import call_dmget

In [ ]:
# Define a mode (leave "prod" for now)
mode = "prod"

# Verbosity
verbose = True

# Give your diagnostic a name and a short description
diag_name = "MOC z-space"
diag_desc = "Meridional Overturning Circulation in z-space"

# Define what variables you would like to analyze. The first entry is the
# variable name and the second entry is the realm (post-processing dir).
#   (By default, monthly timeseries data will be loaded. TODO: add documentation
#    on how to select different frequencies, multiple realms to search, etc.)
variables = [
    RequestedVariable("vmo", "ocean_annual_z", frequency="yearly"),
]

# Optional: define runtime settings or options for your diagnostic
user_options = {"regions": ["global", "atlantic"]}

# Initialize the diagnostic with its name, description, vars, and options
diag = NotebookDiagnostic(diag_name, diag_desc, variables=variables, **user_options)

In [ ]:
# Define the groups of experiments to analyze. Provide a single dora id for one experiment
# or a list of IDs to aggregate multiple experiments into one; e.g. historical+future runs
groups = [
    CaseGroup2("odiv-516", date_range=("1993-01-01", "2017-12-31"), name="OM5 B11 NB"),
    CaseGroup2("odiv-290", date_range=("1993-01-01", "2017-12-31"), name="OM5 B01 (OM4-like)"),
    CaseGroup2("odiv-554", date_range=("0041-01-01", "0060-12-31"), name="CM4.0 + OM5 B11"),
    CaseGroup2("odiv-558", date_range=("0041-01-01", "0060-12-31"), name="CM4.0"),
]

In [ ]:
# Combine the experiments with the diag request and determine what files need to be loaded:
diag.resolve(groups)

In [ ]:
# Print a list of file paths
# This cell and the markdown cell that follows are necessary to run this notebook
# Interactively on Dora
_ = [print(x) for x in diag.files]

<i>(The files above are necessary to run the diagnostic.)</i>

In [ ]:
# Check to see the dmget status before calling "open"
call_dmget(diag.files,status=True)
call_dmget(diag.files)

In [ ]:
# Load the data as xarray datasets
diag.open()

## Main Diagnostic

In [ ]:
import xarray as xr
import cmip_basins
import numpy as np
import matplotlib.pyplot as plt
import momgrid as mg

In [ ]:
esnb.sites.gfdl.convert_to_momgrid(diag)

In [ ]:
def calc_moc_z(
    ds,
    varname,
    mask=1.0,
    xdim="xh",
    tdim="time",
    ydim="yq",
    zdim="z_l",
    interfaces="z_i",
):
    arr = ds[varname]
    arr = arr.mean(tdim) * mask
    integ_layers = arr.sum(xdim).reindex({zdim: arr[zdim][::-1]}).cumsum(zdim)
    # The result of the integration over layers is evaluated at the interfaces
    # with psi = 0 as the bottom boundary condition for the integration
    bottom_condition = xr.zeros_like(integ_layers.isel({zdim: 0}))
    # combine bottom condition with data array
    psi_raw = xr.concat(
        [integ_layers.reindex({zdim: integ_layers[zdim][::-1]}), bottom_condition],
        dim=zdim,
    )
    # rename to correct dimension and add correct vertical coordinate
    psi = -psi_raw.rename({zdim: interfaces}).transpose(interfaces, ydim)
    psi[interfaces] = ds[interfaces]
    psi.name = "psi"
    # # Convert kg.s-1 to Sv (1e6 m3.s-1)
    moc_z = psi / rho0 / 1.0e6
    return moc_z

In [ ]:
def plot_panel(ax, y, z, psi, depth, domain):
    if domain == "atlantic":
        levels = np.arange(-28, 30, 2)
    elif domain == "global":
        levels = np.arange(-40, 42, 2)

    cb = ax.contourf(y, z, psi, levels=levels, cmap="RdBu_r")

    # Setting y-axis limits to avoid singular transformation warning for future calls
    ax.set_ylim([np.min(z) - 500, np.max(z) + 500])

    cs = ax.contour(y, z, psi, levels=levels, colors="k", linewidths=0.3)
    zero_contours = ax.contour(y, z, psi, levels=[0], colors="blue", linewidths=0)

    ax.set_yscale("splitscale", zval=[6500.0, 2000.0, 0.0])
    plt.axhline(y=2000, color="k", linestyle="dashed", linewidth=0.8)

    _ = ax.fill_between(y, depth, 6750, color="gray", zorder=2)
    # plt.colorbar(cb)

    stats = {}

    if domain == "atlantic":
        Y, Z = np.meshgrid(y, z)
        mask = (Y >= 20) & (Y <= 80) & (Z >= 500) & (Z <= 2000)
        _psi = psi.values
        max_psi = np.max(_psi[mask])
        max_index = np.unravel_index(np.argmax(_psi[mask]), _psi[mask].shape)
        max_y = Y[mask][max_index]
        max_z = Z[mask][max_index]

        stats = {
            "max": round(float(max_psi), 2),
            "max_lat": round(float(max_y), 2),
            "max_depth": round(float(max_z), 2),
        }

        square_size = 10
        ax.plot(
            max_y,
            max_z,
            marker="s",
            color="yellow",
            markersize=square_size + 5,
            markeredgewidth=2,
            markeredgecolor="yellow",
            markerfacecolor="none",
        )
        ax.annotate(
            f"{max_psi:.1f} Sv",
            xy=(max_y, max_z),
            xytext=(10, 10),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", color="black"),
            fontsize=5,
            bbox=dict(
                facecolor="white",
                edgecolor="black",
                boxstyle="round,pad=0.2",
                linewidth=0.5,
                alpha=0.7,
            ),
        )

        z_values = []
        for path in zero_contours.get_paths():  # Use get_paths() instead of allsegs
            vertices = path.vertices
            x = vertices[:, 0]
            z_val = vertices[:, 1]
            mask = (20 <= x) & (x <= 55) & (2500 <= z_val) & (z_val <= 4000)
            z_values.extend(z_val[mask])

        mean_z = np.mean(z_values) if z_values else None

        if mean_z:
            stats["zero_depth"] = round(float(mean_z), 1)
            ax.annotate(
                f"Mean Depth:\n{mean_z:.0f} m",
                xy=(40, mean_z),
                xytext=(-10, -30),
                textcoords="offset points",
                arrowprops=dict(arrowstyle="->", color="black"),
                fontsize=7,
                bbox=dict(
                    facecolor="white",
                    edgecolor="black",
                    boxstyle="round,pad=0.2",
                    linewidth=0.5,
                    alpha=0.7,
                ),
            )
    return (stats, cb)

In [ ]:
varname = "vmo"
varobj = diag.varmap[varname]

xdim = "xh"
ydim = "yq"
zdim = "z_l"
interfaces = "z_i"
tdim = "time"

rho0 = 1035.0

moc_z_data = {}

for region in diag.settings["diag_vars"]["regions"]:
    moc_z_data[region] = {}
    for group in diag.groups:
        print(f"Calculating {region} moc z for {group.name}")
        ds = group.datasets[varobj]
        deptho = mg.MOMgrid(ds.model, warn=False).to_xarray()["deptho"]
        deptho = xr.DataArray(deptho.values, dims=("yq", "xh"))

        if len(ds[ydim]) != len(deptho[ydim]):
            ds = ds.isel({ydim: slice(1, None)})

        if region == "atlantic":
            # Calculate the basin mask on the v-grid
            basin = cmip_basins.generate_basin_codes(ds, lon="geolon_v", lat="geolat_v")
            mask = xr.where(
                (basin == 2)
                | (basin == 4)
                | (basin == 6)
                | (basin == 7)
                | (basin == 8)
                | (basin == 9),
                1.0,
                np.nan,
            )
        else:
            mask = 1.0

        y = (ds.geolat_v * mask).mean(xdim)
        y = xr.DataArray(y.values, dims=("y"), coords={"y": y.values})
        yloc = xr.where(y.isnull(), False, True)

        z = ds[interfaces]

        psi = calc_moc_z(ds, varname, mask=mask)
        psi = xr.DataArray(
            psi.values, dims=("z", "y"), coords={"z": z.values, "y": y.values}
        )
        psi = psi[:, yloc]

        depth = xr.DataArray(
            (deptho * mask).max("xh").values, dims=("y"), coords={"y": y.values}
        )
        depth = depth[yloc]

        y = y[yloc]

        moc_z_data[region][group] = (y, z, psi, depth, region)

In [ ]:
all_figs = []

esnb.nbtools.setup_plots()

for region in diag.settings["diag_vars"]["regions"]:
    nexps = len(diag.groups)
    figsize, subplot = esnb.nbtools.get_figsize_subplots(nexps)
    figsize = (5.41 * 1.25, 3.35 * 0.75 * 2)
    fig = plt.figure(figsize=figsize, dpi=200)

    axes = []
    for n, group in enumerate(diag.groups):
        ax = plt.subplot(*subplot, n + 1)
        stats, cb = plot_panel(ax, *moc_z_data[region][group])
        ax.set_title(group.name)
        axes.append(ax)

        if len(stats) > 0:
            for k, v in stats.items():
                group.add_metric(region, (k, float(v)))

    if region == "global":
        for ax in axes:
            ax.set_xlim(-78, None)

    plt.subplots_adjust(wspace=0.3)

    cbar = esnb.nbtools.bottom_colorbar(
        fig, cb, orientation="horizontal", extend="both"
    )
    cbar.set_label(f"{str(region).capitalize()} Meridional Streamfunction [Sv]")

    # add letter labels for each panel
    esnb.nbtools.panel_letters(axes, -0.12, 1.05)

In [ ]:
diag.write_metrics("MOC_z_metrics.json")